<a href="https://colab.research.google.com/github/silver-or/dacon/blob/main/%ED%8C%94%EB%8B%B9%EB%8C%90_%ED%99%8D%EC%88%98_%EC%95%88%EC%A0%84%EC%9A%B4%EC%98%81%EC%97%90_%EB%94%B0%EB%A5%B8_%ED%95%9C%EA%B0%95_%EC%88%98%EC%9C%84%EC%98%88%EC%B8%A1_%EB%B2%A0%EC%9D%B4%EC%8A%A4%EB%9D%BC%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 필요한 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np

from glob import glob
from tqdm import tqdm
from scipy import interpolate

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, AveragePooling1D, GlobalAveragePooling1D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 데이터 전처리

In [ ]:
w_list = sorted(glob(r"/content/drive/MyDrive/colab/competition_data/water_data/*.csv"))
w_list

['/content/drive/MyDrive/colab/competition_data/water_data/data_2012.csv',
 '/content/drive/MyDrive/colab/competition_data/water_data/data_2013.csv',
 '/content/drive/MyDrive/colab/competition_data/water_data/data_2014.csv',
 '/content/drive/MyDrive/colab/competition_data/water_data/data_2015.csv',
 '/content/drive/MyDrive/colab/competition_data/water_data/data_2016.csv',
 '/content/drive/MyDrive/colab/competition_data/water_data/data_2017.csv',
 '/content/drive/MyDrive/colab/competition_data/water_data/data_2018.csv',
 '/content/drive/MyDrive/colab/competition_data/water_data/data_2019.csv',
 '/content/drive/MyDrive/colab/competition_data/water_data/data_2020.csv',
 '/content/drive/MyDrive/colab/competition_data/water_data/data_2021.csv',
 '/content/drive/MyDrive/colab/competition_data/water_data/data_2022.csv']

In [ ]:
pd.read_csv(w_list[0]).shape

(26496, 15)

In [ ]:
pd.read_csv(w_list[0]).head(4)

,ymdhm,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630
0,2012-05-01 00:00,24.800,555.0,219.07,24.93,555.0,445.0,310.7,469.05,300.2,0.0,290.0,729.80,275.3,540.18
1,2012-05-01 00:10,24.794,464.6,218.86,25.15,562.9,449.0,314.7,498.00,300.2,0.0,290.0,731.48,275.3,540.18
2,2012-05-01 00:20,24.789,478.1,218.69,25.31,576.4,451.0,313.7,490.68,301.2,0.0,290.0,726.42,275.3,540.18
3,2012-05-01 00:30,24.789,464.8,218.69,25.31,563.1,452.0,311.7,476.21,301.2,0.0,290.0,726.42,276.3,552.17


In [ ]:
train_data = []
train_label = []
num = 0

for i in w_list[:-1]:
  tmp = pd.read_csv(i)
  tmp = tmp.replace(" ", np.nan)
  tmp = tmp.interpolate(method = 'values')  # 선형으로 비례하여 결측값 보간
  tmp = tmp.fillna(0)

  # 431은 3일 간격, 432는 예측하고자 하는 시간
  for j in tqdm(range(len(tmp)-432)):
    train_data.append(np.array(tmp.loc[j: j+431, ["swl", "inf", "sfw", "ecpc",
                                "tototf", "tide_level",
                                "fw_1018662", "fw_1018680",
                                "fw_1018683", "fw_1019630"]]).astype(float))
    train_label.append(np.array(tmp.loc[j + 432, ["wl_1018662", "wl_1018680",
                                                  "wl_1018683", "wl_1019630"]]).astype(float))

100%|██████████| 26064/26064 [00:26<00:00, 989.98it/s]


In [ ]:
train_data = np.array(train_data)
train_label = np.array(train_label)

print(train_data.shape)
print(train_label.shape)

(260640, 432, 10)
(260640, 4)


### 모델링 및 모델 학습

In [ ]:
input_shape = (train_data[0].shape[0], train_data[0].shape[1])

model = Sequential()
model.add(GRU(256, input_shape=input_shape))
model.add(Dense(4, activation='relu'))

optimizer = tf.optimizers.RMSprop(0.001)

model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

In [17]:
model.fit(train_data, train_label, epochs=10, batch_size=128)

Epoch 1/10
2037/2037 [==============================] - 86s 42ms/step - loss: 248.5114 - mae: 7.9789
Epoch 2/10
2037/2037 [==============================] - 85s 42ms/step - loss: 248.5723 - mae: 8.1448
Epoch 3/10
2037/2037 [==============================] - 84s 41ms/step - loss: 251.8332 - mae: 8.2945
Epoch 4/10
2037/2037 [==============================] - 85s 42ms/step - loss: 248.6523 - mae: 8.2934
Epoch 5/10
2037/2037 [==============================] - 85s 42ms/step - loss: 269.3845 - mae: 8.6577
Epoch 6/10
2037/2037 [==============================] - 85s 42ms/step - loss: 257.7566 - mae: 8.7302
Epoch 7/10
2037/2037 [==============================] - 85s 42ms/step - loss: 328.5203 - mae: 9.2913
Epoch 8/10
2037/2037 [==============================] - 84s 41ms/step - loss: 303.0479 - mae: 9.3615
Epoch 9/10
2037/2037 [==============================] - 84s 41ms/step - loss: 284.6966 - mae: 9.1706
Epoch 10/10
2037/2037 [==============================] - 85s 41ms/step - loss: 269.3258 - m

### 추론 데이터셋 만들기

In [19]:
test_data = []
test_label = []

tmp = pd.read_csv(w_list[-1])
tmp = tmp.replace(" ", np.nan)

# 이전 값을 사용 (결측값을 앞 방향으로 채우기)
tmp = tmp.fillna(method = 'pad')
tmp = tmp.fillna(0)

# tmp.loc[:, ["wl_1018662", "wl_1018680", "wl_1018683", "wl_1019630"]] = tmp.loc[:, ["wl_1018662", "wl_1018680", "wl_1018683", "wl_1019630"]]*100

for j in tqdm(range(4032, len(tmp)-432)):
    test_data.append(np.array(tmp.loc[j:j + 431, ["swl", "inf", "sfw", "ecpc",
                                                    "tototf", "tide_level",
                                                    "fw_1018662", "fw_1018680",
                                                    "fw_1018683", "fw_1019630"]]).astype(float))
        
    test_label.append(np.array(tmp.loc[j + 432, ["wl_1018662", "wl_1018680",
                                                    "wl_1018683", "wl_1019630"]]).astype(float))

100%|██████████| 6912/6912 [00:08<00:00, 776.62it/s]


In [20]:
test_data = np.array(test_data)
test_label = np.array(test_label)

print(test_data.shape)
print(test_label.shape)

(6912, 432, 10)
(6912, 4)


In [21]:
score = model.evaluate(test_data, test_label)
print('Test score:', score[0])
print('Test accuracy:', score[1])

216/216 [==============================] - 2s 9ms/step - loss: 108646.4531 - mae: 321.1088
Test score: 108646.453125
Test accuracy: 321.1087951660156


### 제출 파일 만들기

In [22]:
pred = model.predict(test_data)

In [23]:
pred = pd.DataFrame(pred)

In [24]:
sample_submission = pd.read_csv("/content/drive/MyDrive/colab/competition_data/sample_submission.csv")

sample_submission["wl_1018662"] = pred[0]
sample_submission["wl_1018680"] = pred[1]
sample_submission["wl_1018683"] = pred[2]
sample_submission["wl_1019630"] = pred[3]

In [26]:
sample_submission.to_csv("/content/drive/MyDrive/colab/competition_data/baseline.csv", index = False)